#### Создание и обучение сети

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

SIZE = 128
DATA_DIR = 'data'

In [ ]:
pairs = np.load(f'{DATA_DIR}/pairs.npy', mmap_mode='r+')
labels = np.load(f'{DATA_DIR}/labels.npy', mmap_mode='r+')

In [ ]:
pairs.shape, labels.shape

In [ ]:
x = pairs[:, 0]  # Первое изображение
y = pairs[:, 1]  # Второе изображение

Разделение на обучающую и тестовую выборку

In [ ]:
X_train_pairs, X_test_pairs, y_train_pairs, y_test_pairs = train_test_split(pairs, labels, test_size=0.1, random_state=42)

X_train_pairs.shape, y_train_pairs.shape, X_test_pairs.shape, y_test_pairs.shape

#### Создаем модель

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model

img_a_inp = Input((SIZE, SIZE), name='img_a_inp')
img_b_inp = Input((SIZE, SIZE), name='img_b_inp')


def get_cnn_block(depth):
    return Sequential([Conv2D(depth, 3, 1),
                       BatchNormalization(),
                       ReLU()])


DEPTH = 64
cnn = Sequential([Reshape((SIZE, SIZE, 1)),
                  get_cnn_block(DEPTH),
                  get_cnn_block(DEPTH),
                  get_cnn_block(DEPTH),
                  GlobalAveragePooling2D(),
                  Dropout(0.2),
                  Flatten(),
                  Dense(400, activation='relu')])


feature_vector_A = cnn(img_a_inp)
feature_vector_B = cnn(img_b_inp)

concat = Concatenate()([feature_vector_A, feature_vector_B])

dense = Dense(200, activation='relu')(concat)
dropout = Dropout(0.5)(dense)
output = Dense(1, activation='sigmoid')(dropout)

model = Model(inputs=[img_a_inp, img_b_inp], outputs=output)

model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

Функция просмотра процесса обучения

Запуск через терминал `tensorboard --logdir=logs`

In [ ]:
from tensorflow.python.keras.callbacks import TensorBoard

call = [TensorBoard(log_dir='logs', histogram_freq=1, write_images=True)]

Функция обнаружения переобучения

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=False,)

Поиск оптимальных значений batch_size

In [ ]:
from itertools import chain
divs = lambda n: chain(*((d, n // d) for d in range(1, int(n ** 0.5) + 1) if n % d == 0))
print(sorted(list(divs(X_train_pairs.shape[0]))))

In [ ]:
batch_size = 36

#### Обучение

При обучении используем только обучающую выборку
с помощью validation_split разделяем ее на обучающую и валидационную

In [ ]:
model.fit(x=[X_train_pairs[:, 0, :, :], X_train_pairs[:, 1, :, :]],
          y=y_train_pairs,
          validation_split=0.25,
          epochs=100,
          batch_size=batch_size,
          callbacks=[es, call])

print('Обучение остановлено на эпохе', es.stopped_epoch)

In [ ]:
model.save(f'model')

#### Тест

In [ ]:
from tensorflow.python.keras.models import load_model

model = load_model(f'model')

In [ ]:
X_test_pairs.shape, y_test_pairs.shape

In [ ]:
from random import randint

right_answer = 0
wrong_answer = 0

for _ in range(100):
    x = randint(0, X_test_pairs.shape[0] - 1)

    img_A, img_B = X_test_pairs[x, 0, :, :], X_test_pairs[x, 1, :, :]
    label = y_test_pairs[x]

    pred = model.predict([img_A.reshape((1, SIZE, SIZE)),
                          img_B.reshape((1, SIZE, SIZE))]).flatten()[0] > 0.5

    if (label and pred) or (not label and not pred):
        right_answer += 1
    else:
        wrong_answer += 1

percent = int(100 / ((right_answer + wrong_answer) / right_answer))

print(f'Процент правильных ответов: {percent}')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

x = randint(0, X_test_pairs.shape[0] - 1)

img_A, img_B = X_test_pairs[x, 0, :, :], X_test_pairs[x, 1, :, :]
print(model.predict([img_A.reshape((1, 128, 128)),
                     img_B.reshape((1, 128, 128))]).flatten()[0] > 0.5)

pic_box = plt.figure(figsize=(32, 16))
for i, picture in enumerate([img_A, img_B]):
    pic_box.add_subplot(2, 5, i + 1)
    plt.imshow(picture, cmap='gray')
    plt.axis('off')
plt.show()